In [1]:
import os
import numpy
import mmcv
import torch
import matplotlib.pyplot as plt
import sklearn
import pickle
import pandas
import gzip
import sys
import plotly_express as px
import plotly.graph_objects as go
sys.path.insert(0, os.path.abspath('../../'))
from marrovision.cortex.data.bone_marrow.utilities import get_results_comparison_table
import os
import torch
import torch.distributed
import argparse
import logging
import marrovision.cortex.data as data_lib
import marrovision.cortex.model as model_lib
import marrovision.cortex.trainer as trainer_lib
from marrovision.utilities.checkpointing.get_start_epoch import get_start_epoch
from marrovision.utilities.device import get_device
from marrovision.utilities.io.files_and_folders import clean_folder
from marrovision.utilities.randomization.seed import fix_random_seeds
from marrovision.contrib.mmcv import Config
from marrovision.utilities.argument_parsing.train import train_parser
from marrovision.cortex.data.bone_marrow.transformations import eval_transform_1

2022-03-18 15:05:40,814 - apex.transformer.tensor_parallel - WARNING - `fused_weight_gradient_mlp_cuda` module not found. gradient accumulation fusion with weight gradient computation disabled.


In [2]:
config_path = os.path.abspath(os.path.join('../configs/resnet/resnext50_32x4d.py'))

In [4]:
config = dict(Config.fromfile(config_path))

In [6]:
device = torch.device('cpu')

In [7]:
print(f"~> preparing the model (model type: {config['model']['type']})...\n")
model = getattr(model_lib, config['model']['type'])(
    config=config['model']['config'],
    device=device
).to(device)

~> preparing the model (model type: ClassifierWithTorchvisionBackbone)...



In [13]:
ckpt = torch.load(os.path.join(config['trainer']['config']['checkpointing']['repo'], 'ckpt_latest.pth'), map_location='cpu')
stats = torch.load(os.path.join(config['trainer']['config']['checkpointing']['repo'], 'stats_latest-rank0.pth'), map_location='cpu')
label_layout = stats['test'][0]['label_layout']['labels']
model.load_state_dict(ckpt['model'])

In [46]:
img = mmcv.imread('/data/marrovision/BM_cytomorphology_data/BAS/BAS_00001.jpg', channel_order='rgb')

In [47]:
[label_layout[e] for e in model.forward({'image': eval_transform_1()(img).unsqueeze(0), 'label_index': torch.zeros(1).long()}, mode='test')['model_outputs']['y_hat'].tolist()][0]

'BAS'